In [40]:
import os
import re

from pyproj import Transformer
from pystac.extensions.projection import ProjectionExtension
import pystac.item
from shapely.geometry import GeometryCollection, box, shape, mapping
from datetime import datetime, UTC
import pystac

import rasterio as rio
import rasterio.warp
import rasterio.crs

import xml.etree.cElementTree as ET
from xml.dom import minidom

In [41]:
img_path = 'X:/EO/u2018_clc2012_v2020_20u1_raster100m/DATA/U2018_CLC2012_V2020_20u1.tif'

id = os.path.basename(img_path).split('.')[0]
_ = re.search('(?<=CLC)[0-9]{4}', id)
year = _.group(0)

# Wrap in def?

props = {'description': (f'Corine Land Cover {year} (CLC{year}) is one of the Corine Land Cover (CLC) ' 
                         f'datasets produced within the frame the Copernicus Land Monitoring Service '
                         f'referring to land cover / land use status of year {year}. '
                         f'CLC service has a long-time heritage (formerly known as \"CORINE Land Cover Programme\"), '
                         f'coordinated by the European Environment Agency (EEA). It provides consistent '
                         f'and thematically detailed information on land cover and land cover changes across Europe. '
                         f'CLC datasets are based on the classification of satellite images produced by the national '
                         f'teams of the participating countries - the EEA members and cooperating countries (EEA39). '
                         f'National CLC inventories are then further integrated into a seamless land cover map of Europe. '
                         f'The resulting European database relies on standard methodology and nomenclature with following '
                         f'base parameters: 44 classes in the hierarchical 3-level CLC nomenclature; '
                         f'minimum mapping unit (MMU) for status layers is 25 hectares; '
                         f'minimum width of linear elements is 100 metres. '
                         f'Change layers have higher resolution, i.e. minimum mapping unit (MMU) is 5 hectares '
                         f'for Land Cover Changes (LCC), and the minimum width of linear elements is 100 metres. '
                         f'The CLC service delivers important data sets supporting the implementation of key priority '
                         f'areas of the Environment Action Programmes of the European Union as e.g. protecting ecosystems, '
                         f'halting the loss of biological diversity, tracking the impacts of climate change, '
                         f'monitoring urban land take, assessing developments in agriculture or dealing with '
                         f'water resources directives. CLC belongs to the Pan-European component of the '
                         f'Copernicus Land Monitoring Service (https://land.copernicus.eu/), part of the '
                         f'European Copernicus Programme coordinated by the European Environment Agency, '
                         f'providing environmental information from a combination of air- and space-based observation '
                         f'systems and in-situ monitoring. Additional information about CLC product description including '
                         f'mapping guides can be found at https://land.copernicus.eu/user-corner/technical-library/. '
                         f'CLC class descriptions can be found at '
                         f'https://land.copernicus.eu/user-corner/technical-library/corine-land-cover-nomenclature-guidelines/html/.'),
         'created': None,
         'providers': [{
                        'name': 'Copernicus Land Monitoring Service',
                        'description': ('The Copernicus Land Monitoring Service provides '
                                        'geographical information on land cover and its '
                                        'changes, land use, ground motions, vegetation state, '
                                        'water cycle and Earth\'s surface energy variables to '
                                        'a broad range of users in Europe and across the World '
                                        'in the field of environmental terrestrial applications.'),
                        'roles': ['licensor', 'host'],
                        'url': 'https://land.copernicus.eu'
                      }],
         


}



with rio.open(img_path) as img:

    # xmin, ymin, xmax, ymax = img.bounds
    # transformer = Transformer.from_crs(img.crs, "EPSG:4326")
    # xmin_, ymax_ = transformer.transform(xmin, ymax)
    # xmax_, ymin_ = transformer.transform(xmax, ymin)
    # box_ = box(*[xmin_, ymin_, xmin_, ymax_])
    # box_ = box(*transformer.transform_bounds(*img.bounds))
    # bounds_wgs84 = rasterio.warp.transform_bounds(img.crs, CRS.from_epsg(4326), *img.bounds)
    bbox = rio.warp.transform_bounds(img.crs, rio.crs.CRS.from_epsg(4326), *img.bounds)
    params = {
        'id': id,
        'bbox': bbox,
        'geometry': mapping(box(*bbox)),
        'datetime': None,
        'start_datetime': datetime(int(year), 1, 1, microsecond=0, tzinfo=UTC),
        'end_datetime': datetime(int(year), 12, 31, microsecond=0, tzinfo=UTC),
        'properties': props,
    }



item = pystac.Item(**params)


item.add_asset(
    key='image',
    asset=pystac.Asset(href=img_path, title='Geotiff', media_type=pystac.MediaType.GEOTIFF),
)

# item.ext.add('proj')

In [42]:
from pystac.extensions.projection import ProjectionExtension
import pystac.link

proj_ext = ProjectionExtension.ext(item.assets['image'], add_if_missing=True)

proj_ext.apply(epsg=rio.crs.CRS(img.crs).to_epsg(),
               bbox=img.bounds,
               shape=[_ for _ in img.shape],
               transform=[_ for _ in img.transform] + [0.0, 0.0, 1.0],
               )

license = pystac.link.Link(rel='LICENSE', target="https://land.copernicus.eu/en/data-policy")
item.add_link(license)



In [43]:
item.save_object(dest_href='testY.json')

In [28]:
datetime(int(year), 1, 1, microsecond=0)

datetime.datetime(2012, 1, 1, 0, 0)

In [17]:
# Taken from https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary
from xml.etree import cElementTree as ElementTree


class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                else:
                    aDict = {element[0].tag: XmlListConfig(element)}
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            elif element.items():
                self.update({element.tag: dict(element.items())})
            else:
                self.update({element.tag: element.text})

stac_io = pystac.StacIO.default()

def get_metadata(xml: str):
    result = XmlDictConfig(ElementTree.XML(stac_io.read_text(xml)))
    result[
        "ORIGINAL_URL"
    ] = xml  # Include the original URL in the metadata for use later
    return result

In [18]:
xml_path = '../CLC_samples/U2018_CLC2018_V2020_20u1.xml'

get_metadata(xml_path)


{'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.isotc211.org/2005/gmd http://schemas.opengis.net/csw/2.0.2/profiles/apiso/1.0.0/apiso.xsd',
 '{http://www.isotc211.org/2005/gmd}fileIdentifier': {'{http://www.isotc211.org/2005/gco}CharacterString': '7e162b2d-5196-41b2-b6dd-e889651e2f1f'},
 '{http://www.isotc211.org/2005/gmd}language': {'{http://www.isotc211.org/2005/gmd}LanguageCode': {'codeList': 'http://www.loc.gov/standards/iso639-2/',
   'codeListValue': 'eng'}},
 '{http://www.isotc211.org/2005/gmd}characterSet': {'{http://www.isotc211.org/2005/gmd}MD_CharacterSetCode': {'codeList': 'http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CharacterSetCode',
   'codeListValue': 'utf8'}},
 '{http://www.isotc211.org/2005/gmd}hierarchyLevel': {'{http://www.isotc211.org/2005/gmd}MD_ScopeCode': {'codeList': 'http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_ScopeCode',
   'codeListValue': 'dataset'}},
 '{http://www.isotc211.org/2005/gmd}con

In [9]:
def xml_to_dict(xml, result):
    for child in xml:
        if len(child) == 0:
            result[child.tag] = child.text
        else:
            if child.tag in result:
                if not isinstance(result[child.tag], list):
                    result[child.tag] = [result[child.tag]]
                result[child.tag].append(xml_to_dict(child, {}))
            else:
                result[child.tag] = xml_to_dict(child, {})
    return result


In [10]:
xml_to_dict(tree.getroot(), {})

{'{http://www.isotc211.org/2005/gmd}fileIdentifier': {'{http://www.isotc211.org/2005/gco}CharacterString': '7e162b2d-5196-41b2-b6dd-e889651e2f1f'},
 '{http://www.isotc211.org/2005/gmd}language': {'{http://www.isotc211.org/2005/gmd}LanguageCode': None},
 '{http://www.isotc211.org/2005/gmd}characterSet': {'{http://www.isotc211.org/2005/gmd}MD_CharacterSetCode': None},
 '{http://www.isotc211.org/2005/gmd}hierarchyLevel': {'{http://www.isotc211.org/2005/gmd}MD_ScopeCode': None},
 '{http://www.isotc211.org/2005/gmd}contact': {'{http://www.isotc211.org/2005/gmd}CI_ResponsibleParty': {'{http://www.isotc211.org/2005/gmd}organisationName': {'{http://www.isotc211.org/2005/gco}CharacterString': 'European Environment Agency'},
   '{http://www.isotc211.org/2005/gmd}contactInfo': {'{http://www.isotc211.org/2005/gmd}CI_Contact': {'{http://www.isotc211.org/2005/gmd}address': {'{http://www.isotc211.org/2005/gmd}CI_Address': {'{http://www.isotc211.org/2005/gmd}deliveryPoint': {'{http://www.isotc211.org/

In [118]:
# bbox coordinates become meaningless...

crs = rio.crs.CRS(img.crs)
bounds = img.bounds

transformer = Transformer.from_crs(crs.to_epsg(), 4326)
miny, minx = transformer.transform(bounds.left, bounds.bottom)
maxy, maxx = transformer.transform(bounds.right, bounds.top)
bbox = (minx, miny, maxx, maxy)

rio.coords.BoundingBox(*bbox)


BoundingBox(left=-23.825993823627275, bottom=24.28417701147754, right=104.35721125172725, top=80.00331843607006)

In [105]:
crs = CRS(img.crs)
bounds = img.bounds

transformer = Transformer.from_crs(crs.to_epsg(), 4326)
miny, minx = transformer.transform(bounds.left, bounds.bottom)
maxy, maxx = transformer.transform(bounds.right, bounds.top)
bbox = (miny, minx, maxy, maxx)
# bbox = (minx, miny, maxx, maxy)
rio.coords.BoundingBox(*bbox)

BoundingBox(left=24.28417701147754, bottom=-23.825993823627275, right=80.00331843607006, top=104.35721125172725)

In [98]:
# wrong order...
from pyproj import Transformer

transformer = Transformer.from_crs(img.crs.to_epsg(), "EPSG:4326")
rio.coords.BoundingBox(*transformer.transform_bounds(*img.bounds))

BoundingBox(left=24.28417701147754, bottom=-180.0, right=90.0, top=180.0)

In [119]:
# correct order...
from rasterio.crs import CRS

rio.coords.BoundingBox(*rio.warp.transform_bounds(img.crs, CRS.from_epsg(4326), *img.bounds))

BoundingBox(left=-56.50514190170437, bottom=24.28417701147754, right=72.90613675900903, top=72.63376966542347)